# Feature Extraction

In [1]:
import os
import glob
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

In [2]:
features = pd.DataFrame()

## Traversing through files to extract features

In [3]:
f_list = []
path = './test_files'
for filename in glob.glob(os.path.join(path, '*.csv')):
    df = pd.read_csv(filename)
    dphi = np.diff(np.asarray(df['lat'].values, dtype = np.float64))
    dlambda = np.diff(np.asarray(df['lon'].values, dtype = np.float64))
    d2alt = np.diff(np.diff(np.asarray(df['alt'].values, dtype = np.float64)))
    dt = np.diff(np.asarray(df['t'].values, dtype = np.float64))
    dphi /= dt
    dlambda /= dt
    d2alt /= dt[:-1]
         
    dalt_mean = np.mean(d2alt)
    
    threshold_for_maneuver = 30
    is_maneuver = 0
    print (np.max(np.abs(d2alt)))
    if np.max(np.abs(d2alt)) > threshold_for_maneuver:
        is_maneuver = 1
    plane_speed = np.sqrt(np.power(dphi, 2) + np.power(dlambda, 2)) 
    
    plane_speed *= 111000
    
    dist = ((df['lat'].values[-1] - df['lat'].values[0]) ** 2 +
        (df['lon'].values[-1] - df['lon'].values[0]) ** 2)
    dist = np.sqrt(dist)
    
    f_list.append([filename, np.max(np.asarray(df['alt'].values, 
            dtype = np.float64)), dalt_mean,
            np.mean(plane_speed), dist, is_maneuver])

813.4739056032413
0.0
10.000000000014552
15.022958443625612
654.904534982571


In [4]:
len(f_list)

5

In [5]:
f_list = np.asarray(f_list)

In [7]:
for idx, x in enumerate(['fname', 'alt_max', 'vert_acc', 'horiz_speed', 'dist', 'is_maneuver']):
    features[x] = f_list[:, idx]

In [8]:
features.describe()

,fname,alt_max,vert_acc,horiz_speed,dist,is_maneuver
count,5,5,5,5,5,5
unique,5,5,5,5,5,2
top,./test_files/traj21.csv,8067.8996402249695,0.0,2807.9897656181306,0.13132728153315604,0
freq,1,1,1,1,1,3


In [9]:
features = features.sort_values(['fname'], ascending = [1], axis = 0)

## Radar Cross Section Features Included

In [10]:
rcs_mean = [80, 3, 0.01, 0.1, 15]

In [11]:
rcs_values = []
for i in rcs_mean:
    for k in np.random.normal(i, i/10, 1):
        rcs_values.append(k)

In [ ]:
len(rcs_values)

In [12]:
rcs_values

[91.83784268013125,
 2.8218944430413226,
 0.01036623202065583,
 0.1047093708129697,
 13.527321733401044]

In [13]:
features['rcs'] = rcs_values

In [14]:
features.to_csv('test_features.csv', index = False)

In [ ]:
features